In [36]:
import mne
import numpy as np
from matplotlib import pyplot as plt
import math
from scipy import stats, signal
import itertools
import pandas as pd
from datetime import datetime
import os

In [ ]:
%matplotlib qt
raw = mne.io.read_raw_edf('C:\\Lilach\\402_for_tag.edf')
raw.plot(duration=30, scalings='auto')

In [ ]:
for x in raw.annotations:
    print(x)

In [ ]:
# raw.annotations.save('C:\\Lilach\\402_anot.csv')

In [3]:
# Get tags from annotations (only until the 'end' cause it's 100% covered)
raw = mne.io.read_raw_edf('C:\\Lilach\\402_for_tag.edf')

df = pd.DataFrame(raw.annotations)
i = df.loc[df['description'] == 'END'].index[0]
df = df[:i]
right = df[df['description'].str.contains('Rt')]
left = df[df['description'].str.contains('Lt')]

Extracting EDF parameters from C:\Lilach\402_for_tag.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [ ]:
path = 'C:\\Users\\user\\PycharmProjects\\pythonProject\\results\\'
subj = '402'
right_tags = pd.read_csv(path + f'{subj}\\{subj}_tags.csv')['R'].dropna()

""


In [5]:

np.arange(60, 241, 60)

array([ 60, 120, 180, 240])

In [21]:
# from mnelab.io.writers import write_edf
# for subj in ['396', '398', '402', '405', '406', '415', '416']:
#     raw = mne.io.read_raw_edf(f'C:\\Lilach\\{subj}_for_tag.edf')
#     raw.load_data()
#     raw.filter(l_freq=0.1, h_freq=500)
#     raw.notch_filter((60, 120, 180, 240), method='spectrum_fit')
#     write_edf(f'C:\\Lilach\\{subj}_for_tag_filtered.edf', raw)

In [25]:
df = pd.read_csv("C:\\analysis\\402\\best_thresh_detect\\402_LAH1_t5_6_10.csv")

In [26]:
df

,Unnamed: 0,threshold_type,first_index,last_index,max_index,max_amp,duration,duration_sec
0,1,"['env', 'grad', 'amp']",4751,4849,4833,0.000534,98,0.0490
1,2,"['env', 'grad']",1346,1375,1375,0.000090,29,0.0145
2,3,"['env', 'grad']",4751,4829,4829,0.000522,78,0.0390
3,4,env,4755,4776,4767,0.000286,21,0.0105
4,5,grad,26667,26677,26667,-0.000106,10,0.0050
...,...,...,...,...,...,...,...,...
2341,2342,env,47260861,47260891,47260863,0.000161,30,0.0150
2342,2343,"['env', 'grad', 'amp']",47286026,47286138,47286103,0.000653,112,0.0560
2343,2344,"['env', 'grad']",47286026,47286096,47286096,0.000631,70,0.0350
2344,2345,env,47286026,47286078,47286078,0.000320,52,0.0260


In [27]:
df.sample(100)

,Unnamed: 0,threshold_type,first_index,last_index,max_index,max_amp,duration,duration_sec
59,60,"['env', 'grad']",1623218,1623300,1623300,0.000645,82,0.0410
1963,1964,"['env', 'grad']",40529289,40529360,40529360,0.000207,71,0.0355
49,50,"['env', 'grad', 'amp']",1508616,1508714,1508691,0.000610,98,0.0490
546,547,"['env', 'grad']",12624643,12624710,12624686,-0.000169,67,0.0335
1626,1627,env,31962924,31962971,31962958,0.000144,47,0.0235
...,...,...,...,...,...,...,...,...
931,932,grad,20161862,20161872,20161872,0.000035,10,0.0050
1567,1568,"['env', 'grad']",31240934,31240978,31240978,0.000781,44,0.0220
145,146,"['env', 'grad']",2402419,2402446,2402422,-0.000115,27,0.0135
784,785,amp,16157557,16157585,16157570,0.000225,28,0.0140


In [32]:
'amp' in 'env'

False

In [30]:
'amp' in ['env', 'grad']

False

In [57]:
# create files with 100 samples
channel_name = 'LAH1'
#for subj in ['396', '398', '402', '406', '415', '416']:
for subj in ['396']:
    raw = mne.io.read_raw_fif(f'C:\\UCLA\\{subj}_cz+bi_full_filtered.fif')
    sf = raw.info['sfreq']
    path = f'C:\\analysis\\{subj}\\best_thresh_detect'
    pred_file = [x for x in os.listdir(path) if '2' not in x and 'tag' not in x][0]
    spikes_df = pd.read_csv(path + '\\' + pred_file)
    stim_amp, stim_grad, stim_env = np.zeros((1, raw.n_times)), np.zeros((1, raw.n_times)), np.zeros((1, raw.n_times))
    for evt in spikes_df.sample(100).values:
        if 'amp' in evt[1]:
            stim_amp[0][evt[4]] = 1
        if 'grad' in evt[1]:
            stim_grad[0][evt[4]] = 1
        if 'env' in evt[1]:
            stim_env[0][evt[4]] = 1

    raw.pick_channels([x for x in raw.ch_names if x[0] == 'L' and '-' not in x])
    info_amp = mne.create_info(['AMP'], raw.info['sfreq'], ['stim'])
    info_grad = mne.create_info(['GRAD'], raw.info['sfreq'], ['stim'])
    info_env = mne.create_info(['ENV'], raw.info['sfreq'], ['stim'])
    amp_raw = mne.io.RawArray(stim_amp, info_amp)
    grad_raw = mne.io.RawArray(stim_grad, info_grad)
    env_raw = mne.io.RawArray(stim_env, info_env)
    raw.load_data()
    raw.add_channels([amp_raw, grad_raw, env_raw], force_update_info=True)
    raw.save(f'C:\\analysis\\{subj}\\{subj}_LAH1_100_samples.fif', overwrite=True)


Opening raw data file C:\UCLA\396_cz+bi_full_filtered.fif...


C:\Users\user\AppData\Local\Temp/ipykernel_13268/286939407.py:5: RuntimeWarning: This filename (C:\UCLA\396_cz+bi_full_filtered.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(f'C:\\UCLA\\{subj}_cz+bi_full_filtered.fif')


Isotrak not found
    Range : 0 ... 17883999 =      0.000 ...  8941.999 secs
Ready.
Opening raw data file C:\UCLA\396_cz+bi_full_filtered-1.fif...
Isotrak not found
    Range : 17884000 ... 35767999 =   8942.000 ... 17884.000 secs
Ready.
Opening raw data file C:\UCLA\396_cz+bi_full_filtered-2.fif...
Isotrak not found
    Range : 35768000 ... 50401999 =  17884.000 ... 25201.000 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=50402000
    Range : 0 ... 50401999 =      0.000 ... 25201.000 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=50402000
    Range : 0 ... 50401999 =      0.000 ... 25201.000 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=50402000
    Range : 0 ... 50401999 =      0.000 ... 25201.000 secs
Ready.
Reading 0 ... 50401999  =      0.000 ... 25201.000 secs...
Writing C:\analysis\396\396_LAH1_100_samples.fif


C:\Users\user\AppData\Local\Temp/ipykernel_13268/286939407.py:28: RuntimeWarning: This filename (C:\analysis\396\396_LAH1_100_samples.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(f'C:\\analysis\\{subj}\\{subj}_LAH1_100_samples.fif', overwrite=True)


Writing C:\analysis\396\396_LAH1_100_samples-1.fif
Closing C:\analysis\396\396_LAH1_100_samples-1.fif
Closing C:\analysis\396\396_LAH1_100_samples.fif
[done]


In [60]:
spikes_df.sample(100)

,Unnamed: 0,threshold_type,first_index,last_index,max_index,max_amp,duration,duration_sec
106,107,env,2029731,2029747,2029740,0.000321,16,0.0080
2577,2578,amp,46573247,46573283,46573269,0.001822,36,0.0180
1086,1087,"['env', 'grad']",18340520,18340597,18340565,-0.001534,77,0.0385
1384,1385,env,22667435,22667461,22667443,0.000183,26,0.0130
2712,2713,env,49215548,49215569,49215554,0.000214,21,0.0105
...,...,...,...,...,...,...,...,...
2333,2334,grad,41159214,41159248,41159214,-0.001426,34,0.0170
1549,1550,"['env', 'grad', 'amp']",27198741,27198813,27198782,0.001426,72,0.0360
2013,2014,"['env', 'grad']",36156248,36156272,36156264,0.000537,24,0.0120
953,954,"['env', 'grad']",16612243,16612395,16612374,-0.000901,152,0.0760


In [62]:
stim_amp.sum()

9.0

In [65]:
stim_grad.sum()

57.0

In [66]:
stim_env.sum()

84.0

In [67]:
len(spikes_df)

2785

In [68]:
len(spikes_df[spikes_df['threshold_type'].str.contains(',')])

1339